# 🎬 Tech Challenge Fase 4 - Versão Simplificada

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mavsousa30/techchallengefase4/blob/main/notebooks/Colab_Simple.ipynb)

## ⚡ IMPORTANTE:
1. **Ative a GPU**: Runtime → Change runtime type → GPU (T4)
2. **Execute TODAS as células em ordem**
3. Aguarde downloads na primeira execução (~2-3 min)

## 📋 O que este notebook faz:
- ✅ Detecta rostos
- ✅ Classifica emoções
- ✅ Reconhece atividades
- ✅ Gera relatórios

**Tempo estimado**: 5-10 min para vídeo de 1 minuto

## 1️⃣ Setup Completo (Execute esta célula primeiro)

In [ ]:
%%capture
# Instalar dependências
!pip install -q opencv-python numpy pandas deepface torch torchvision tqdm pydantic rich

# Clonar repositório
!rm -rf techchallengefase4
!git clone https://github.com/Mavsousa30/techchallengefase4.git

# Mudar para o diretório PRIMEIRO
%cd techchallengefase4

# DEPOIS criar diretórios (no lugar certo)
import os
os.makedirs('outputs', exist_ok=True)
os.makedirs('outputs/logs', exist_ok=True)
os.makedirs('outputs/frames', exist_ok=True)

print('✅ Setup concluído!')
print(f'📂 Diretório: {os.getcwd()}')
print(f'📁 outputs/ existe: {os.path.exists("outputs")}')

In [ ]:
# Verificar GPU e recursos
import torch
import psutil

print('🎮 GPU:')
if torch.cuda.is_available():
    print(f'   ✅ {torch.cuda.get_device_name(0)}')
    print(f'   💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
else:
    print('   ⚠️  Nenhuma GPU - Será mais lento!')
    print('   💡 Ative: Runtime → Change runtime type → GPU')

print(f'\n💾 RAM: {psutil.virtual_memory().total / 1e9:.1f} GB')
print(f'🖥️  CPU: {psutil.cpu_count()} cores')
print(f'\n📂 Diretório: {os.getcwd()}')

## 2️⃣ Upload do Vídeo

In [ ]:
from google.colab import files

print('📤 Selecione seu vídeo (MP4, AVI, MOV, MKV):')
uploaded = files.upload()

video_file = list(uploaded.keys())[0]
video_size = len(uploaded[video_file]) / (1024**2)

print(f'\n✅ Upload concluído!')
print(f'📹 Arquivo: {video_file}')
print(f'📊 Tamanho: {video_size:.2f} MB')

if video_size > 100:
    print('\n⚠️  Vídeo grande! Processamento pode demorar.')
    print('💡 Dica: Use vídeos < 2 minutos para testes rápidos')

## 3️⃣ Processar Vídeo

### 🔧 Nota sobre Codec de Vídeo

Se encontrar erro `VideoWriterError: Failed to write frame`, pode ser um problema de codec.
O notebook usa `mp4v` por padrão, que funciona na maioria dos casos.

**Se tiver problemas, não se preocupe** - os relatórios JSON e MD serão gerados normalmente!

### 🧪 Diagnóstico (Execute se tiver problemas)

Se encontrar erro de VideoWriter, execute a célula abaixo para diagnosticar:

In [ ]:
# 🧪 DIAGNÓSTICO COMPLETO - Execute se tiver erro VideoWriterError
!python test_videowriter_colab.py

In [ ]:
# Importar módulos
import sys
sys.path.insert(0, '/content/techchallengefase4')

from src.pipeline.inference import InferencePipeline
from src.metrics.reporter import Reporter

print('✅ Módulos importados!')
print('🎬 Iniciando processamento...\n')

In [ ]:
# 🔍 VERIFICAÇÃO: Garantir que tudo está no lugar certo
import os

print('🔍 Verificando ambiente...')
print(f'📂 Diretório atual: {os.getcwd()}')
print(f'📁 outputs/ existe: {os.path.exists("outputs")}')
print(f'📁 src/ existe: {os.path.exists("src")}')

# Criar outputs se não existir (garantia extra)
if not os.path.exists('outputs'):
    print('⚠️  Criando diretório outputs...')
    os.makedirs('outputs', exist_ok=True)
    os.makedirs('outputs/logs', exist_ok=True)
    os.makedirs('outputs/frames', exist_ok=True)

# Testar se consegue escrever
test_file = 'outputs/test_write.txt'
try:
    with open(test_file, 'w') as f:
        f.write('test')
    os.remove(test_file)
    print('✅ Permissões de escrita OK!')
except Exception as e:
    print(f'❌ Erro ao escrever: {e}')

print('\n✅ Ambiente verificado! Pronto para processar.')

In [ ]:
# Criar e executar pipeline
pipeline = InferencePipeline(
    video_path=video_file,
    output_video_path='outputs/annotated.mp4',
    save_preview=True,
    face_backend='opencv',
    emotion_backend='deepface'
)

# Processar (pode demorar alguns minutos)
summary = pipeline.run()

print('\n🎉 Processamento concluído!')

In [ ]:
# 🔧 ALTERNATIVA: Se o vídeo anotado der erro, use esta célula
# Processa SEM salvar vídeo (apenas gera métricas e relatórios)

# pipeline = InferencePipeline(
#     video_path=video_file,
#     output_video_path=None,  # Não salvar vídeo
#     save_preview=False,       # Desabilitar preview
#     face_backend='opencv',
#     emotion_backend='deepface'
# )
# 
# summary = pipeline.run()
# print('\n✅ Processamento concluído (sem vídeo anotado)!')

## 4️⃣ Resultados

In [ ]:
# Mostrar resumo
print('📊 RESUMO DO PROCESSAMENTO')
print('=' * 60)
print(f'📹 Vídeo: {summary["video_path"]}')
print(f'🎞️  Frames: {summary["frames_total"]:,}')
print(f'⏱️  Duração: {summary["duration_seconds"]:.2f}s')
print(f'🎯 FPS: {summary["fps"]:.2f}')
print(f'\n👤 FACES')
print(f'   Total: {summary["faces_stats"]["total"]}')
print(f'   Média/frame: {summary["faces_stats"]["mean_per_frame"]:.2f}')
print(f'   Máximo: {summary["faces_stats"]["max_in_frame"]}')

if summary['emotions_distribution']:
    print(f'\n😊 EMOÇÕES')
    for emotion, count in sorted(summary['emotions_distribution'].items(), 
                                   key=lambda x: x[1], reverse=True)[:5]:
        print(f'   {emotion}: {count}')

if summary['activities_timeline']:
    print(f'\n🎭 ATIVIDADES: {len(summary["activities_timeline"])} eventos')

print(f'\n⚠️  ANOMALIAS: {summary["anomalies_total"]}')
print('=' * 60)

In [ ]:
# Visualizar vídeo anotado
from IPython.display import Video

print('🎬 Vídeo com Anotações:')
Video('outputs/annotated.mp4', width=800, height=600)

## 5️⃣ Gerar Relatórios

In [ ]:
# Gerar relatórios
reporter = Reporter()
reporter.save_metrics(summary, 'outputs/metrics.json')
reporter.generate_markdown_report(summary, 'outputs/report.md')

print('✅ Relatórios gerados:')
print('   📄 outputs/metrics.json')
print('   📄 outputs/report.md')
print('   🎬 outputs/annotated.mp4')

## 6️⃣ Download dos Resultados

In [ ]:
from google.colab import files
import zipfile

# Criar ZIP
zip_file = 'resultados.zip'
with zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write('outputs/annotated.mp4', 'annotated.mp4')
    zipf.write('outputs/metrics.json', 'metrics.json')
    zipf.write('outputs/report.md', 'report.md')

import os
size = os.path.getsize(zip_file) / (1024**2)
print(f'📦 Arquivo criado: {zip_file} ({size:.2f} MB)')
print('⬇️  Iniciando download...')

files.download(zip_file)
print('✅ Download concluído!')

## 📊 Análise Extra (Opcional)

Gráficos de emoções e atividades

In [ ]:
import matplotlib.pyplot as plt

# Gráfico de emoções
if summary['emotions_distribution']:
    emotions = dict(sorted(summary['emotions_distribution'].items(), 
                           key=lambda x: x[1], reverse=True)[:7])
    
    fig, ax = plt.subplots(figsize=(12, 6))
    bars = ax.bar(emotions.keys(), emotions.values(), color='skyblue', edgecolor='navy')
    
    # Adicionar valores nas barras
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',
                ha='center', va='bottom', fontweight='bold')
    
    ax.set_xlabel('Emoção', fontsize=12, fontweight='bold')
    ax.set_ylabel('Frequência', fontsize=12, fontweight='bold')
    ax.set_title('Distribuição de Emoções Detectadas', fontsize=14, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print('⚠️  Nenhuma emoção detectada')

---

## ✅ Concluído!

### 📚 Recursos:
- [Repositório GitHub](https://github.com/Mavsousa30/techchallengefase4)
- [Documentação](https://github.com/Mavsousa30/techchallengefase4/blob/main/README.md)
- [Troubleshooting](https://github.com/Mavsousa30/techchallengefase4/blob/main/notebooks/TROUBLESHOOTING_COLAB.md)

### 💡 Dicas:
- Para processar outro vídeo, execute apenas as células 2-6
- Use GPU (T4) para melhor performance
- Vídeos curtos (< 2 min) são processados mais rápido

**🎉 Obrigado por usar o Tech Challenge Fase 4!**